In [0]:
# importing libraries
import re
import pandas as pd
import requests
from urllib.request import urlopen as ureq
from requests import get
from bs4 import BeautifulSoup as BS
import time
import numpy as np
from urllib.parse import urlparse
from tqdm import tqdm_notebook as tqdm

In [0]:
#Get Data Funtion

def get_dataset(link, league):
  df = pd.DataFrame(columns=['Year', 'Link'])

  r = requests.get(link)
  soup = BS(r.content,"html5lib")
  rows = soup.find('select', attrs={'name':'saison'}).find_all('option')
  for row in rows:
    link = row['value']
    year = row.text.strip()
    if year != '2019/2020':
      df = df.append(pd.Series([str(year), link], index=df.columns ), ignore_index=True)
  if league == 'Serie A':
    df = df[:31]
  else:
    df = df[:29]

  dataset = pd.DataFrame(columns=['Player', 'Country', 'League', '1990/1991',
   '1991/1992',
   '1992/1993',
   '1993/1994',
   '1994/1995',
   '1995/1996',
   '1996/1997',
   '1997/1998',
   '1998/1999',
   '1999/2000',
   '2000/2001',
   '2001/2002',
   '2002/2003',
   '2003/2004',
   '2004/2005',
   '2005/2006',
   '2006/2007',
   '2007/2008',
   '2008/2009',
   '2009/2010',
   '2010/2011',
   '2011/2012',
   '2012/2013',
   '2013/2014',
   '2014/2015',
   '2015/2016',
   '2016/2017',
   '2017/2018',
   '2018/2019'])

  for i in range(0,len(df['Year'])):
        r = requests.get('https://www.worldfootball.net'+str(df['Link'][i]))
        soup = BS(r.content,"html5lib") 
        rows = soup.find("table", attrs={'class':'standard_tabelle'}).find("tbody").find_all("tr")
        for row in rows[1:]:
          player = row.find('td').find_next('td').text.strip()
          country = row.find('td').find_next('td').find_next('td').find_next('td').text.strip()
          goals = row.find('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').text.strip()
          goals = goals.split('(')[0]
          if player not in set(dataset['Player']):
            row_num = len(dataset['Player'])
            dataset.loc[row_num,'Player'] = player
            dataset.loc[row_num,'Country'] = country          
            dataset.loc[row_num,'League'] = league
          else:
            row_num = dataset.index[dataset['Player'] == player][0]
          dataset.loc[row_num,'%s' % df['Year'][i]] = goals
  return dataset

In [0]:
epl_data = get_dataset('https://www.worldfootball.net/goalgetter/eng-premier-league-2018-2019/', 'EPL')
laliga_data = get_dataset('https://www.worldfootball.net/goalgetter/esp-primera-division-2018-2019/', 'La Liga')
ligue1_data = get_dataset('https://www.worldfootball.net/goalgetter/fra-ligue-1-2018-2019/', 'Ligue 1')
serie_a_data = get_dataset('https://www.worldfootball.net/goalgetter/ita-serie-a-2018-2019/', 'Serie A')
bundesliga_data = get_dataset('https://www.worldfootball.net/goalgetter/bundesliga-2018-2019/', 'Bundesliga')

In [0]:
epl_data.fillna(0, inplace=True)
laliga_data.fillna(0, inplace=True)
ligue1_data.fillna(0, inplace=True)
serie_a_data.fillna(0, inplace=True)
bundesliga_data.fillna(0, inplace=True)

serie_a_data.drop(['1999/2000 Playoff','1998/1999 Uefa-Cup'], axis=1, inplace=True)

In [0]:
epl_data.to_csv('Goals in EPL (1990-2019).csv', index=False)
laliga_data.to_csv('Goals in La Liga (1990-2019).csv', index=False)
ligue1_data.to_csv('Goals in Ligue 1 (1990-2019).csv', index=False)
serie_a_data.to_csv('Goals in Serie A (1990-2019).csv', index=False)
bundesliga_data.to_csv('Goals in Bundesliga (1990-2019).csv', index=False)

In [0]:
combined = pd.concat([epl_data, laliga_data, ligue1_data, serie_a_data, bundesliga_data], axis=0, ignore_index=True)
cols = combined.columns.drop(['Player', 'Country', 'League'])
combined[cols] = combined[cols].apply(pd.to_numeric, errors='coerce')

In [0]:
group = combined.groupby('Player')['1990/1991',
   '1991/1992',
   '1992/1993',
   '1993/1994',
   '1994/1995',
   '1995/1996',
   '1996/1997',
   '1997/1998',
   '1998/1999',
   '1999/2000',
   '2000/2001',
   '2001/2002',
   '2002/2003',
   '2003/2004',
   '2004/2005',
   '2005/2006',
   '2006/2007',
   '2007/2008',
   '2008/2009',
   '2009/2010',
   '2010/2011',
   '2011/2012',
   '2012/2013',
   '2013/2014',
   '2014/2015',
   '2015/2016',
   '2016/2017',
   '2017/2018',
   '2018/2019'].agg('sum').reset_index()

In [213]:
group.loc[group['Player']=='Cristiano Ronaldo']

,Player,1990/1991,1991/1992,1992/1993,1993/1994,1994/1995,1995/1996,1996/1997,1997/1998,1998/1999,1999/2000,2000/2001,2001/2002,2002/2003,2003/2004,2004/2005,2005/2006,2006/2007,2007/2008,2008/2009,2009/2010,2010/2011,2011/2012,2012/2013,2013/2014,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019
1726,Cristiano Ronaldo,0,0,0,0,0,0,0,0,0,0,0,0,0,4,5,9,17,31,18,26,40,46,34,31,48,35,25,26,21


In [0]:
group.to_csv('Goals in Top 5 Leagues (1990-2019).csv', index=False)

In [0]:
group_country = combined.groupby('Country')['1990/1991',
   '1991/1992',
   '1992/1993',
   '1993/1994',
   '1994/1995',
   '1995/1996',
   '1996/1997',
   '1997/1998',
   '1998/1999',
   '1999/2000',
   '2000/2001',
   '2001/2002',
   '2002/2003',
   '2003/2004',
   '2004/2005',
   '2005/2006',
   '2006/2007',
   '2007/2008',
   '2008/2009',
   '2009/2010',
   '2010/2011',
   '2011/2012',
   '2012/2013',
   '2013/2014',
   '2014/2015',
   '2015/2016',
   '2016/2017',
   '2017/2018',
   '2018/2019'].agg('sum').reset_index()

In [0]:
group_country.to_csv('Goals in Top 5 Leagues by Country(1990-2019).csv', index=False)